In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import time
import logging
# Load environment variables from api.env file
load_dotenv('api.env')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
def upload_file(path):
    # Upload a file with an "assistants" purpose
    file = client.files.create(file=open(path, "rb"), purpose="assistants")
    return file


In [85]:
file = client.files.create(file=open('reskillarticle.pdf', "rb"), purpose="assistants")

In [86]:
file.id

'file-qSyZo22N8pW0moOtTz4xLKJz'

In [87]:
def create_assistant(file):
    """
    You currently cannot set the temperature for Assistant via the API.
    """
    assistant = client.beta.assistants.create(
        name="read_document_assistant",
        instructions="Use your knowledge base to best respond to customer queries. If you don't know the answer. Be friendly and funny.",
        tools=[{"type": "retrieval"}],
        model="gpt-4-1106-preview",
        file_ids=[file.id],
    )
    return assistant

In [91]:
#def thread_creation():
thread = client.beta.threads.create()
thread_id = thread.id
thread_id

'thread_qRJlnKqyEPOWYn2sO09Fb5mX'

In [60]:
file = upload_file("reskillarticle.pdf")

In [88]:
assistant = create_assistant(file)

In [90]:
assistant.id

'asst_7kNEB15cYd0OIuLcRtz4epzk'

In [64]:
message_body = "what skills are priority to be reskilled?"

In [46]:
def generate_respond(message_body):
    message = client.beta.threads.messages.create(
        thread_id = thread_id,
        role = 'user',
        content = message_body
        
)

In [92]:
def run_assistant():
    assistant = client.beta.assistants.retrieve("asst_7kNEB15cYd0OIuLcRtz4epzk")
    thread = client.beta.threads.retrieve('thread_qRJlnKqyEPOWYn2sO09Fb5mX')

    run = client.beta.threads.runs.create(
        thread_id = thread.id,
        assistant_id = assistant.id
    )
    while run.status != "completed":
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id = thread.id, run_id = run.id)

    messages = client.beta.threads.messages.list(thread_id=thread.id)
    new_message = messages.data[0].content[0].text.value
    logging.info(f'generated message: {new_message}')
    return new_message

In [70]:
new_message = run_assistant()

In [71]:
new_message

"I noticed you've sent a file my way! How can I assist you with it? Need a quick find or some info from inside? Just let me know! 😊✨"

In [72]:
print(new_message)

I noticed you've sent a file my way! How can I assist you with it? Need a quick find or some info from inside? Just let me know! 😊✨


In [93]:
new_mess = generate_respond('summarize my file')

In [94]:
new_message = run_assistant()
new_message

"I'm sorry, I attempted to open and summarize your file, but it seems that the file had no contents. There may have been an issue with the upload. Could you please try uploading the file again? If it's too large, perhaps you could provide a specific section or topic that you would like summarized, and I can help with that part."

In [95]:
assistant

Assistant(id='asst_7kNEB15cYd0OIuLcRtz4epzk', created_at=1704820226, description=None, file_ids=['file-qSyZo22N8pW0moOtTz4xLKJz'], instructions="Use your knowledge base to best respond to customer queries. If you don't know the answer. Be friendly and funny.", metadata={}, model='gpt-4-1106-preview', name='read_document_assistant', object='assistant', tools=[ToolRetrieval(type='retrieval')])

In [106]:
thread

Thread(id='thread_bDVAY8SAAbzUp9iVWIYBiyTy', created_at=1704820650, metadata={}, object='thread')

In [97]:
file

FileObject(id='file-qSyZo22N8pW0moOtTz4xLKJz', bytes=147046, created_at=1704820156, filename='reskillarticle.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [98]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "summarize this file",
      "file_ids": [file.id]
    }
  ]
)

In [100]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  model="gpt-4-1106-preview",
  instructions="summarize this file",
  tools=[ {"type": "retrieval"}]
)

In [105]:
run

Run(id='run_IvGyF5j2UbmeN6QjZiwp267z', assistant_id='asst_7kNEB15cYd0OIuLcRtz4epzk', cancelled_at=None, completed_at=None, created_at=1704820805, expires_at=1704821405, failed_at=None, file_ids=['file-qSyZo22N8pW0moOtTz4xLKJz'], instructions='summarize this file', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_bDVAY8SAAbzUp9iVWIYBiyTy', tools=[ToolAssistantToolsRetrieval(type='retrieval')])

In [103]:
file

FileObject(id='file-qSyZo22N8pW0moOtTz4xLKJz', bytes=147046, created_at=1704820156, filename='reskillarticle.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [ ]:
message = client.beta.threads.messages.retrieve(
  thread_id="thread_bDVAY8SAAbzUp9iVWIYBiyTy",
  message_id="..."
)